# pySAS v1.4.8 --> v2.0
<hr style="border: 2px solid #fadbac" />

- **Description:** Differences between pySAS v1.4 and v2.0.
- **Credit:** Ryan Tanner (October 2025)
- **Support:** <a href="https://heasarc.gsfc.nasa.gov/docs/xmm/xmm_helpdesk.html">XMM Newton GOF Helpdesk</a>
- **Last verified to run:** 15 October 2025

<hr style="border: 2px solid #fadbac" />

If you are upgrading pySAS from v1.4.2-v1.4.8 (also known as the development versions) to v2.0, this next section is a quick explanation for the major differences. This includes versions of pySAS:
1. Installed using `pip`
2. Installed from GitHub
3. pySAS on SciServer

If you have been using the "standard" version of pySAS (v1.4.1) distributed with SAS, or pySAS on ESA's Datalab, skip down to the section `Using pySAS v2.0`.

## ObsID and MyTask

Key changes:
  - `ObsID` is the new `ODFobject`
  - The `Wrapper` has been deprecated and replaced by `MyTask`
  - `MyTask` now accepts the options `output_to_terminal` and `output_to_file`
  - Input arguments can be passed in as either a dictionary or a list

To implement these changes make the following modifications to your code:

Change:
```python
odf = pysas.odfcontrol.ODFobject(obsid)
```
to
```python
my_obs = pysas.ObsID(obsid)
```

And change:
```python
from pysas.wrapper import Wrapper as w
w('Task Name',inargs).run()
```
to
```python
from pysas import MyTask
MyTask('Task Name',inargs).run()
```

Everything else should work the same.

### ObsID

`ObsID` works just like `ODFobject` and contains all the same methods, plus a few new ones. You can still do basic setup for a single Obs ID by running:
```python
obsid = '0123456789'
my_obs = pysas.ObsID(obsid)
my_obs.basic_setup()
```

### MyTask
`MyTask` replaces the `Wrapper`, and works in the same way, but with new inputs allowed. These are (with defaults):
```python
logfilename = None, 
tasklogdir  = None,
output_to_terminal = True, 
output_to_file     = False
```
These are the same as the inputs for `ObsID`, though `logfilename` will default to the task name, and `tasklogdir` will default to the current working directory (which should be the `work_dir` since that is where you will be running SAS tasks).

<hr style="border: 2px solid #fadbac" />

# Using pySAS v2.0

`ObsID` contains basic methods and classes for interacting with XMM data files. It is **not** a new SAS task, but it can make doing basic or common analysis easier. Almost everything is contained in a single Python class called `ObsID`.

## obsid.ObsID

To start analysing a single observation the following lines can be used (insert the particular Obs ID you will be using):
```python
import pysas
obsid  = '0123456789'
my_obs = pysas.ObsID(obsid)
my_obs.basic_setup()
```
That will cover set up for 80-90% of use cases. `basic_setup` will assume you are working with `ODF` data, and will only download `ODF` data. If you want to use `PPS` data you will need to use either `download_PPS_data` or `download_ALL_data`.

The `ObsID` class accepts inputs to control output logging. The inputs (with defaults) to `ObsID` are:
```python
obsid (required)
data_dir    = None
logfilename = None
tasklogdir  = None
output_to_terminal = True
output_to_file     = False
```
  - `obsid` is required and has to be the 10-digit observation ID number for the observation you are working with.
  - `data_dir` is the directory where you want the XMM data downloaded.
  - `logfilename`, if this is defined, then all output will be written to this file (but only if `output_to_file=True`). If no file name is given then the name of the log file will be 'ObsID_'+the Obs ID you are working with. Any SAS tasks run using `basic_setup` (i.e. `cifbuild`, `odfingest`, `emproc`, `epproc`, and `rgsproc`) will have their output written to their own file in the `work_dir`.
  - `tasklogdir` is the directory where output log files will be written. If not defined then it will use the `data_dir` for all top level Python related output, and `work_dir` for all other SAS tasks.
  - `output_to_terminal`, if `True` then output will be written to the terminal, if `False` then not.
  - `output_to_file`, if `True` then output will be written to a log file, if `False` then not.

`ObsID` contains the following methods:

```python
basic_setup(data_dir    = None,
            repo        = None,
            overwrite   = False,
            rerun       = False,
            recalibrate = False,
            run_epproc  = True,
            run_emproc  = True,
            run_rgsproc = True,
            run_epchain = False,
            run_emchain = False,
            **kwargs)

run_MyTask(taskname, inargs, **kwargs)

quick_eplot(fits_event_list_file,
            image_file = 'image.fits',
            ximagesize = '600',
            yimagesize = '600',
            vmin = 1.0,
            vmax = 10.0,
            **kwargs)

quick_lcplot(fits_event_list_file,
             light_curve_file = 'light_curve.fits',
             timebinsize      = '100',
             **kwargs)

download_ODF_data(repo        = 'esa',
                  data_dir    = None,
                  overwrite   = False,
                  proprietary = False,
                  credentials_file = None,
                  encryption_key   = None)

download_PPS_data(repo      = 'esa',
                  data_dir  = None,
                  overwrite = False,
                  proprietary      = False,
                  credentials_file = None,
                  encryption_key   = None,
                  PPS_subset   = False,
                  instname     = None,
                  expflag      = None,
                  expno        = None,
                  product_type = None,
                  datasubsetno = None,
                  sourceno     = None,
                  extension    = None,
                  filename     = None,
                  **kwargs)

download_ALL_data(repo        = 'esa',
                  data_dir    = None,
                  overwrite   = True,
                  proprietary      = False,
                  credentials_file = None,
                  encryption_key   = None)

calibrate_odf(obs_dir = None,
              sas_ccf = None,
              sas_odf = None,
              cifbuild_opts  = {},
              odfingest_opts = {},
              recalibrate    = False)

get_active_instruments()

sas_talk(verbosity=4,suppress_warning=1)

find_event_list_files(print_output=True)

find_rgs_spectra_files(print_output=True)

check_for_ccf_cif()

check_for_SUM_SAS()
```

<div class="alert alert-block alert-info">
    <b>Note:</b> Some of these functions are not intended to be used by the typical pySAS user. Internal pySAS use only.
</div>

The only functions desinged to be used by the typical pySAS user are:

    - basic_setup
    - run_MyTask
    - quick_eplot
    - quick_lcplot
    - download_PPS_data
    - sas_talk

Other functions that can be used by pySAS users, but usually you will not need to, are:

    - calibrate_odf
    - download_ODF_data
    - download_ALL_data
    - get_active_instruments
    - find_event_list_files
    - find_rgs_spectra_files
    - check_for_ccf_cif
    - check_for_SUM_SAS

<hr style="border: 2px solid #fadbac" />

When a user creates an `ObsID` object using:
```python
obsid = '##########'
my_obs = pysas.obsid.ObsID(obsid)
```
the `ODF` object will have a few important links and information by default. These include:

    - odfid:    The ObsID number used to create the ODF object.
    - data_dir: Path to base data directory where all XMM data files 
                will be downloaded.
    - obs_dir:  Path to directory containing the data files 
                (obs_dir = data_dir/obsid).
    - odf_dir:  Path to directory with the raw observation data files 
                (odf_dir = obs_dir/ODF).
    - work_dir: The working directory when SAS tasks will be run 
                (work_dir = obs_dir/work).
    - sas_ccf:  Link to the 'ccf.cif' file, if it exists. By default the 
                'ccf.cif' file will be created in the work_dir.
    - sas_odf:  Link to the '*SUM.SAS' file, if it exists. By default the 
                '*SUM.SAS' file will be created in the work_dir.
    - If they exist, links to the basic event lists will be stored in a 
      dictionary named 'files'.

If the ODF files have previously been downloaded and are in the default data directory, then upon creating the ObsID object (i.e. "my_obs = pysas.obsid.ObsID(obsid)") pySAS will automatically find and link all important summary and calibration files, and also all previously generated event lists made by `epproc`, `emproc`, and `rgsproc`.

In [1]:
import pysas

<hr style="border: 2px solid #fadbac" />

### ObsID

In [2]:
pysas.ObsID?

Init signature:
pysas.ObsID(
    obsid,
    data_dir=None,
    logfilename=None,
    tasklogdir=None,
    output_to_terminal=True,
    output_to_file=False,
)
Docstring:     
Class for and Obs ID object.
Inputs:
Required:
    - obsid: 10 digit number of the Obs ID

Optional:
    - data_dir   : Data directory. If none is given, 
                   will use (in this order):
                   1. data_dir set in configuration file
                   2. Current directory
    - logfilename: Name of log file where all output
                   will be written. Overrides default
                   log file names.
    - tasklogdir : Directory for log files. Overrides
                   default log directory.
    - output_to_terminal: If True, then logger information
                          will be output to the terminal.
    - output_to_file: If True, then logger information will
                      be written to a log file.
File:           /opt/envs/sas/lib/python3.12/site-packages/pysas/

## Documentation on ObsID methods

In [3]:
pysas.ObsID.basic_setup?

Signature:
pysas.ObsID.basic_setup(
    self,
    data_dir=None,
    repo=None,
    overwrite=False,
    rerun=False,
    recalibrate=False,
    run_epproc=True,
    run_emproc=True,
    run_rgsproc=True,
    run_epchain=False,
    run_emchain=False,
    **kwargs,
)
Docstring:
Function to do all basic analysis tasks. The function will:

    1. Download data by calling 'download_data'
    2. Call the function 'calibrate_odf'
        A. Run 'cifbuild'
        B. Run 'odfingest'
    2. Run 'epproc' -OR- 'epchain'
    3. Run 'emproc' -OR- 'emchain'
    4. Run 'rgsproc'

If 'run_epchain' is set to 'True', then 'epproc' will not run.
If 'run_emchain' is set to 'True', then 'emproc' will not run.

Inputs:

    data_dir:    Data directory.
    repo:        Download repository ('esa','heasarc','fornax','aws').
    overwrite:   Remove previous data files and download again.
    rerun:       Rerun the *procs or *chains.
    recalibrate: Rerun 'cifbuild' and 'odfingest'.

All input arguments for '

In [4]:
pysas.ObsID.run_MyTask?

Signature: pysas.ObsID.run_MyTask(self, taskname, inargs=None, **kwargs)
Docstring:
This acts as a wrapper around 'MyTask'. This provides a way of calling
SAS tasks, while using the values set when the 'ObsID' object was 
instantiated.

Required inputs (just like MyTask):
    taskname
    inargs

Optional inputs (just like MyTask, but **only** use these if you want 
them to be different from the values used when instantiating 'ObsID'):
    logfilename
    tasklogdir
    output_to_terminal
    output_to_file
    logger (Only in very rare circumstances **DO NOT USE** unless you
            know exactly what you are doing!!)
File:      /opt/envs/sas/lib/python3.12/site-packages/pysas/obsid/obsid.py
Type:      function

In [5]:
pysas.ObsID.quick_eplot?

Signature:
pysas.ObsID.quick_eplot(
    self,
    fits_event_list_file,
    image_file='image.fits',
    xcolumn='X',
    ycolumn='Y',
    ximagesize='600',
    yimagesize='600',
    expression=None,
    vmin=1.0,
    vmax=10.0,
    **kwargs,
)
Docstring:
Quick plot function for EPIC event lists. As input takes an 
event list and uses 'evselect' to create a FITS image file.

Inputs

(Required)
    fits_event_list_file: Filename of event list.

(Optional)
    image_file: Output filename of the image file.
    xcolumn: FITS file header name for X column data.
    ycolumn: FITS file header name for Y column data.
    ximagesize: Output image X resolution in pixels.
    yimagesize: Output image Y resolution in pixels.
    expression: Filtering expression to be used for 'evselect'.
    vmin: Min value for color map.
    vmax: Max value for color map.
    xlabel: X axis plot label.
    ylabel: Y axis plot label.
    save_file: If set to True, then a .png image of the plot will be saved.
    

In [6]:
pysas.ObsID.quick_lcplot?

Signature:
pysas.ObsID.quick_lcplot(
    self,
    fits_event_list_file,
    light_curve_file='light_curve.fits',
    timebinsize='100',
    **kwargs,
)
Docstring:
Quick plot function to generate a light curve. As input takes an 
event list and uses 'evselect' to create a FITS image file.

All standard inputs to 'MyTask' can be passed in as optional
arguments.
File:      /opt/envs/sas/lib/python3.12/site-packages/pysas/obsid/obsid.py
Type:      function

In [7]:
pysas.ObsID.download_PPS_data?

Signature:
pysas.ObsID.download_PPS_data(
    self,
    repo=None,
    data_dir=None,
    overwrite=False,
    proprietary=False,
    credentials_file=None,
    encryption_key=None,
    PPS_subset=False,
    instname=None,
    expflag=None,
    expno=None,
    product_type=None,
    datasubsetno=None,
    sourceno=None,
    extension=None,
    filename=None,
    **kwargs,
)
Docstring:
This handles preliminary setup for downloading data files, then 
calls download_data (as "dl_data") from sasutils.

Inputs:
    --REQUIRED--

        NONE

    --OPTIONAL--

    --repo:           (string): Which repository to use to download data. 
                                Default: 'esa'
                                Can be either
                                'esa' (data from Europe/ESA) or 
                                'heasarc' (data from North America/NASA) or
                                'aws' (data from AWS s3 bucket (NASA)) or
                                'fornax' (if user is on

In [8]:
pysas.ObsID.download_ALL_data?

Signature:
pysas.ObsID.download_ALL_data(
    self,
    repo=None,
    data_dir=None,
    overwrite=True,
    proprietary=False,
    credentials_file=None,
    encryption_key=None,
)
Docstring:
This function assumes you want to overwrite everything in the
obs_dir. Makes no checks.

This handles preliminary setup for downloading data files, then 
calls download_data (as "dl_data") from sasutils.

Inputs:
    --REQUIRED--

        NONE

    --OPTIONAL--

    --repo:           (string): Which repository to use to download data. 
                                Default: 'esa'
                                Can be either
                                'esa' (data from Europe/ESA) or 
                                'heasarc' (data from North America/NASA) or
                                'aws' (data from AWS s3 bucket (NASA)) or
                                'fornax' (if user is on Fornax)

    --data_dir:  (string/path): Path to directory where the data will be 
                     

In [9]:
pysas.ObsID.sas_talk?

Signature: pysas.ObsID.sas_talk(self, verbosity=4, suppress_warning=1)
Docstring:
Simple function to set general SAS veriables 'verbosity' 
and 'suppress_warning'.
File:      /opt/envs/sas/lib/python3.12/site-packages/pysas/obsid/obsid.py
Type:      function

In [10]:
pysas.ObsID.calibrate_odf?

Signature:
pysas.ObsID.calibrate_odf(
    self,
    obs_dir=None,
    sas_ccf=None,
    sas_odf=None,
    cifbuild_opts={},
    odfingest_opts={},
    recalibrate=False,
)
Docstring:
Before running this function an ObsID object must be created first. e.g.

    my_obs = pysas.obsid.ObsID(obsid)

*Then* the data must be downloaded using:

    my_obs.download_ODF_data()

This function can then be used as, 

    my_obs.calibrate_odf()

If it exists it will search data_dir/obsid and any subdirectories for the ccf.cif
and *SUM.SAS files. Will not rerun calibration if the ccf.cif and *SUM.SAS files
exist, unless recalibrate = True.

Optionally the paths to the ccf.cif and *SUM.SAS files can be given through 
sas_ccf and sas_odf respectively.

Inputs:
    --REQUIRED--

        NONE

    --OPTIONAL--

    --obs_dir:  (string/path): Path to the obs directory. If no path 
                               given, then will look in 
                               data_dir/obsid/. If directory exists t

In [11]:
pysas.ObsID.get_active_instruments?

Signature: pysas.ObsID.get_active_instruments(self)
Docstring:
Checks odf summary file for which instruments were active for that odf.

Assumes that 'sas_odf' already exists and contains the correct path.

Also assumes file name and path are stored in self.files['sas_odf'].
File:      /opt/envs/sas/lib/python3.12/site-packages/pysas/obsid/obsid.py
Type:      function

In [12]:
pysas.ObsID.find_event_list_files?

Signature: pysas.ObsID.find_event_list_files(self, print_output=True)
Docstring:
Checks the observation directory (obs_dir) for basic unfiltered 
event list files created by 'epproc', 'emproc', and 'rgsproc'. 
Stores paths and file names in self.files.

'self.files' is a dictionary with the following keys:

    'PNevt_list'
    'M1evt_list'
    'M2evt_list'
    'R1evt_list'
    'R2evt_list'
File:      /opt/envs/sas/lib/python3.12/site-packages/pysas/obsid/obsid.py
Type:      function

In [13]:
pysas.ObsID.find_rgs_spectra_files?

Signature: pysas.ObsID.find_rgs_spectra_files(self, print_output=True)
Docstring:
Check for RGS spectra files created by rgsproc. Adds them to 
'files' dictrionary with the keys:

    'R1SPEC'
    'R2SPEC'
File:      /opt/envs/sas/lib/python3.12/site-packages/pysas/obsid/obsid.py
Type:      function

In [14]:
pysas.ObsID.clear_obs_dir?

Signature: pysas.ObsID.clear_obs_dir(self)
Docstring: Function to remove all files and subdirectories from the obs_dir.
File:      /opt/envs/sas/lib/python3.12/site-packages/pysas/obsid/obsid.py
Type:      function

In [15]:
pysas.ObsID.clear_work_dir?

Signature: pysas.ObsID.clear_work_dir(self)
Docstring: Function to remove all files and subdirectories from the work_dir.
File:      /opt/envs/sas/lib/python3.12/site-packages/pysas/obsid/obsid.py
Type:      function